# Задание - обучить модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers

In [4]:
!git clone https://github.com/KuzmaKhrabrov/character-tokenizer.git

Cloning into 'character-tokenizer'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20 (delta 5), reused 10 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 5.89 KiB | 2.95 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [5]:
# !pip install transformers

In [6]:
import string
import sys
import os
# char_tokenizer_dir = os.path.abspath('/kaggle/working/character-tokenizer')

sys.path.append('/kaggle/working/character-tokenizer')
from charactertokenizer import CharacterTokenizer

chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"
model_max_length = 64
tokenizer = CharacterTokenizer(chars, model_max_length)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
example = "Привет"
tokens = tokenizer(example)
print(tokens)

{'input_ids': [0, 39, 42, 26, 12, 18, 46, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


Датасет для обучения брал отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip

Задание:  

1. Напишите класс для Dataset/Dataloder и разбейте данные на случайные train / test сплиты в соотношении 50:50.
2. Попробуйте обучить одну или несколько из моделей: Bert, Albert, Deberta. Посчитайте метрику Accuracy на train и test


In [8]:
tokenizer('ящикообр^азный')

{'input_ids': [0, 72, 60, 26, 30, 38, 38, 10, 42, 6, 8, 24, 36, 64, 28, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from torch.utils.data import random_split
import torch
import random

In [10]:
data = pd.read_csv('/kaggle/input/word-stress-dataset/all_accents.tsv', delimiter = '\t',names = ['words','words_with_accent'])
data

,words,words_with_accent
0,-де,-д^е
1,-ка,-к^а
2,-либо,-л^ибо
3,-нибудь,-ниб^удь
4,-с,-с
...,...,...
1680530,ӂюль-верновский,ӂюль-в^ерновский
1680531,ӂюрить,ӂюр^ить
1680532,ӂӂение,ӂӂ^ение
1680533,ӂӂенный,ӂӂенный


In [11]:
data.count(), data.isna().sum()

(words                1680535
 words_with_accent    1680535
 dtype: int64,
 words                0
 words_with_accent    0
 dtype: int64)

### Создадим датасет и даталоадер

Сначала найдем фразу максимальной длины -> трансформер может работать с эмбеддингами одной длины

In [12]:
max_len = 0
wordmax =''
for word in tqdm(data['words']):
    emb = tokenizer.encode(word, add_special_tokens=True)
    if len(word) > max_len:
        
        max_len = len(emb)
        wordmax = word

max_len, wordmax

100%|██████████| 1680535/1680535 [01:14<00:00, 22428.07it/s]


(58, 'лланвайрпуллгуингиллгогерихуирндробуллллантисилиогогогох')

In [13]:
input_ids = []
attention_masks = []
labels = []

for i, word in tqdm(enumerate(data['words'])):
    
    
    labels.append( len(data['words_with_accent'][i].split('^')[0]) )
    
    encoded_word = tokenizer.encode_plus(word,
                         add_special_tokens=True,
                         max_length=max_len,
                         pad_to_max_length=True,
                         return_attention_mask = True,
                         return_tensors = 'pt' 
                         )

    input_ids.append(encoded_word['input_ids'])
    attention_masks.append(encoded_word['attention_mask'])
    
   

print(input_ids[0])
print(attention_masks[0])
print(labels[0])

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1680535it [03:59, 7010.52it/s]

tensor([[ 0,  6, 16, 18,  1,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4]])
tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
2


In [14]:
print(input_ids[0])
print(attention_masks[0])
print(labels[0])


tensor([[ 0,  6, 16, 18,  1,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4]])
tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
2


In [15]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [16]:
from torch.utils.data import random_split, TensorDataset, DataLoader

dataset = TensorDataset(input_ids,attention_masks, labels)

train_size = int(0.5 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [17]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


batch_size = 32

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )


validation_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset), 
            batch_size = batch_size 
        )

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [19]:
from transformers import BertForSequenceClassification, BertConfig


# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-uncased", 
    num_labels = max_len, 
                   
    output_attentions = False, 
    output_hidden_states = False, 
)


if device == "cuda":
    model.cuda()

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

from transformers import get_linear_schedule_with_warmup


epochs = 4


total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Функции для подсчета времени выполнения и метрики accuracy

In [21]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [22]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [24]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []


total_t0 = time.time()

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

   
    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 700 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)

        total_train_loss += output.loss.item()

        output.loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
    
    avg_train_loss = total_train_loss / len(train_dataloader)

    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.save_pretrained('./BERT_word_stress/')

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:

        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            output = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        total_eval_loss += output.loss.item()

        logits = output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

    

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch   700  of  26,259.    Elapsed: 0:02:09.
  Batch 1,400  of  26,259.    Elapsed: 0:04:18.
  Batch 2,100  of  26,259.    Elapsed: 0:06:27.
  Batch 2,800  of  26,259.    Elapsed: 0:08:36.
  Batch 3,500  of  26,259.    Elapsed: 0:10:45.
  Batch 4,200  of  26,259.    Elapsed: 0:12:54.
  Batch 4,900  of  26,259.    Elapsed: 0:15:03.
  Batch 5,600  of  26,259.    Elapsed: 0:17:12.
  Batch 6,300  of  26,259.    Elapsed: 0:19:21.
  Batch 7,000  of  26,259.    Elapsed: 0:21:30.
  Batch 7,700  of  26,259.    Elapsed: 0:23:39.
  Batch 8,400  of  26,259.    Elapsed: 0:25:48.
  Batch 9,100  of  26,259.    Elapsed: 0:27:57.
  Batch 9,800  of  26,259.    Elapsed: 0:30:05.
  Batch 10,500  of  26,259.    Elapsed: 0:32:14.
  Batch 11,200  of  26,259.    Elapsed: 0:34:23.
  Batch 11,900  of  26,259.    Elapsed: 0:36:32.
  Batch 12,600  of  26,259.    Elapsed: 0:38:41.
  Batch 13,300  of  26,259.    Elapsed: 0:40:50.
  Batch 14,000  of  26,259.    Elapsed: 

Таким образом, обучение заняло почти 7 часов на 4 эпохи. Думаю, что с моделью типа Albert обучение бы сильно ускорилось. Однако accuracy получилась отличная - 0.93 